In [20]:
from skimage.filters import threshold_otsu
import os
import glob
import numpy as np
from skimage import io, color, img_as_ubyte
from skimage.filters.rank import entropy
from skimage.morphology import disk
import pandas as pd

def process_images(path):
    time = 0
    time_list = []
    area_list = []
    entropy_img_list = []
    binary_img_list = []
    image_names = []
    
    for time, file in enumerate(glob.glob(path + "/*.*")):
        img = io.imread(file)
        if img.ndim == 3:
            img = color.rgb2gray(img)  # Convert to grayscale if the image is RGB
        entropy_img = entropy(img, disk(22))
        entropy_img = entropy_img * 255.0  # Scale to the range 0-255
        entropy_img = img_as_ubyte(entropy_img)  # Convert entropy_img to uint8
        entropy_img_list.append(entropy_img)
        thresh = threshold_otsu(entropy_img)
        binary_img = entropy_img >= thresh
        binary_img_list.append(binary_img)
        scratch_area = np.sum(binary_img)
        time_list.append(time)
        area_list.append(scratch_area)

        # Extract the image name from the file path and store it in the image_names list
        image_names.append(os.path.basename(file))

    # Create a DataFrame for each time step and concatenate them
    dfs = []
    for t, entropy_img, binary_img in zip(time_list, entropy_img_list, binary_img_list):
        df_time = pd.DataFrame({
            'Time': [t] * entropy_img.size,
            'Area': [np.sum(binary_img)] * entropy_img.size,
            'Image Name': [os.path.basename(file)] * entropy_img.size,
            'Original Image': img.reshape(-1),
            'Entropy': entropy_img.reshape(-1)
        })
        dfs.append(df_time)

    df = pd.concat(dfs, ignore_index=True)

    output_file = 'excel_bp.xlsx'
    df.to_excel(output_file, index=False)

    return time_list, area_list, entropy_img_list, binary_img_list

path_a = "aa"
path_b = "bb"

time_list_a, area_list_a, entropy_img_a, binary_a = process_images(path_a)
time_list_b, area_list_b, entropy_img_b, binary_b = process_images(path_b)


C:\Users\ico\AppData\Local\Temp\ipykernel_1792\346909162.py:58: UserWarning: Possible precision loss converting image of type float64 to uint8 as required by rank filters. Convert manually using skimage.util.img_as_ubyte to silence this warning.
  time_list_a, area_list_a, entropy_img_a, binary_a = process_images(path_a)


ValueError: Images of type float must be between -1 and 1.